In [ ]:
from __future__ import print_function, division
import scipy
from keras.models import load_model
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
from tqdm import tqdm
import keras
import pandas as pd
from keras.datasets import mnist
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import np_utils
import datetime
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
import cv2

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)  
import helpers

# Training file directory
DATASET = os.path.join(ROOT_DIR, 'dataset')

PATH = "{}/{}".format(DATASET, "isic2016numpy")
# load data
x_train = np.load("{}/x_train.npy".format(PATH))
y_train = np.load("{}/y_train.npy".format(PATH))
x_train.shape, y_train.shape

MODEL_PATH = os.path.join(ROOT_DIR, "models")
print(ROOT_DIR)
print(os.listdir(MODEL_PATH))

In [ ]:
# Load translation model
# b2m.h5
model_name = 'generator_isic2016_b2m_100.h5'
model = load_model(os.path.join(MODEL_PATH, model_name), custom_objects={'InstanceNormalization':InstanceNormalization})
model.summary()

In [ ]:
def predict(model, img):
    
    if img.shape[0] != 256:
        print("Resizing image..")
        img = cv2.resize(img, (256, 256))
    
    # Normalize image as the trained distribution
    
    img = img/127.5 - 1.
    
    # Normalize imgae [0, 1]
    #img = img.astype('float32')
    #img /= 255.
    
    img = np.expand_dims(img, axis=0)
    img = model.predict(img)
    img = np.squeeze(img, axis=0)
    
    
    # Rescale to [0,1]
    #img = 0.5 * img + 0.5
    img = (img - np.min(img))/np.ptp(img)
    
    return img


def oversample(x, y, model):
    '''
    Some cool stuff
    INPUT
        x: 
        y:
        model:
        
    OUTPUT
        New folder in the current directory.
    '''
    
    print("Before oversampling :", x.shape, y.shape)
    
    
    # majority class
    majority_samples = []
    for img, label in zip(x, y):
        if label[1] == 0:
            majority_samples.append(img)
        else:
            pass
    
    # numpy array of majority classes
    majority_samples = np.array(majority_samples)
    
    # minority generated samples
    synthetic_samples = []
    
    # iterate over majority samples and generate minority class
    for img in tqdm(majority_samples):
        
        # translate to malignant
        pred = predict(model, img)
        synthetic_samples.append(pred)
    
    # make labels for generated minority classes
    y_syn = np.array([1 for _ in range(len(synthetic_samples))])
    y_syn = np_utils.to_categorical(y_syn, 2)
    
    # Scale training set to [0, 1]
    x = x.astype('float32')
    x /= 255
    
    # merge and shuffle training and generated samples
    x_balanced = np.concatenate( (x, synthetic_samples), axis = 0)
    y_balanced = np.concatenate( (y, y_syn), axis = 0)
    x_balanced, y_balanced = helpers.shuffle_dataset(x_balanced, y_balanced)
    
    assert len(majority_samples) == len(synthetic_samples), "This should be same! If not, check model code"
    assert len(x_balanced) == len(synthetic_samples) + len(x_train), "Check oversampler code"
    print("After oversampling: ", x_balanced.shape, y_balanced.shape)
    
    return majority_samples, synthetic_samples, x_balanced, y_balanced


In [ ]:
raw, gen, x_new, y_new = oversample(x_train, y_train, model)

### Raw data

In [ ]:
from numpy.random import rand
import matplotlib.pyplot as plt

index = np.random.choice(np.array(gen).shape[0], 30, replace=False)
raw = np.array(raw)
x = raw[index]

a, b = 5, 6
x = np.reshape(x, (a, b, 256, 256, 3))


test_data = x
r, c = test_data.shape[0], test_data.shape[1]
cmaps = [['viridis', 'binary'], ['plasma', 'coolwarm'], ['Greens', 'copper']]

heights = [a[0].shape[0] for a in test_data]
widths = [a.shape[1] for a in test_data[0]]

fig_width = 15.  # inches
fig_height = fig_width * sum(heights) / sum(widths)

f, axarr = plt.subplots(r,c, figsize=(fig_width, fig_height),
        gridspec_kw={'height_ratios':heights})

for i in range(r):
    for j in range(c):
        axarr[i, j].imshow(test_data[i][j])
        axarr[i, j].axis('off')
        
plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)
#plt.savefig('{}/{}.png'.format("{}/outputs/".format(ROOT_DIR), "beforegan"), dpi=300)
plt.show()

### Synthesize data

In [ ]:
from numpy.random import rand
import matplotlib.pyplot as plt

gen = np.array(gen)
x = gen[index]

a, b = 5, 6
x = np.reshape(x, (a, b, 256, 256, 3))


test_data = x
r, c = test_data.shape[0], test_data.shape[1]
cmaps = [['viridis', 'binary'], ['plasma', 'coolwarm'], ['Greens', 'copper']]

heights = [a[0].shape[0] for a in test_data]
widths = [a.shape[1] for a in test_data[0]]

fig_width = 15.  # inches
fig_height = fig_width * sum(heights) / sum(widths)

f, axarr = plt.subplots(r,c, figsize=(fig_width, fig_height),
        gridspec_kw={'height_ratios':heights})

for i in range(r):
    for j in range(c):
        axarr[i, j].imshow(test_data[i][j])
        axarr[i, j].axis('off')
        
plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)
#plt.savefig('{}/{}.png'.format("{}/outputs/".format(ROOT_DIR), "aftergan"), dpi=300)
plt.show()

In [ ]:
x_new.shape, y_new.shape

In [ ]:
# Create directory
helpers.create_directory("{}/dataset/isic2016gan/".format(ROOT_DIR))

# Save
np.save("{}/dataset/isic2016gan/{}{}.npy".format(ROOT_DIR, "x_", "upsampled"), x_new)
np.save("{}/dataset/isic2016gan/{}{}.npy".format(ROOT_DIR, "y_", "upsampled"), y_new)

In [ ]:
print("Done!")